In [75]:
# Set up Notebook
% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm
from sklearn.model_selection import GridSearchCV



# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [76]:
train_data = pd.read_csv('tz_train.csv')
test_data = pd.read_csv('tz_test.csv')
train_data = train_data.dropna(axis=1, how='all')
test_data = test_data.dropna(axis=1, how='all')
train_data = train_data.dropna(axis=0, how='any')
test_data = test_data.dropna(axis=0, how='any')


In [77]:
labels = ['logFCS', 'rCSI', 'HDDS']

In [78]:
x_train=train_data.drop(labels, axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_test=test_data.drop(labels, axis=1)
x_test = x_test.drop(x_test.columns[0], axis=1)

x_train.head()

,floor_cement,floor_dirt_sand_dung,roof_iron,roof_natural,slope,ag_percent,dist_road,dist_popcenter,dist_agmkt,dist_headquater,...,maxdaysnorain,dist_km,maize_price,maize_mktthin,rice_price,rice_mktthin,lhz_maize_price,lhz_maize_mktthin,lhz_rice_price,lhz_rice_mktthin
68,0.00,1.00,0.000,1.000,1.675673,84.00,7.7,53.100000,69.900000,8.300000,...,176.0,32.213358,74400.000,0.0,157562.500,0.0,73010.215096,0.000000,162514.557193,0.000000
69,0.00,1.00,0.000,1.000,0.800000,0.00,45.5,50.799999,77.099998,365.899994,...,169.0,32.213358,46379.400,0.0,131016.400,0.0,31771.000000,0.043731,123750.000000,0.043731
70,0.00,1.00,1.000,0.000,1.291615,49.00,13.4,25.500000,63.300000,309.700000,...,176.0,32.213358,54116.000,0.0,127833.360,0.0,43150.839948,0.000000,126015.545902,0.000000
71,0.75,0.25,0.875,0.125,2.721242,9.25,7.0,10.012500,9.437500,255.250000,...,176.0,32.213358,85600.002,0.0,182090.900,0.0,77868.806928,0.000000,183758.272609,0.000000
72,0.00,1.00,1.000,0.000,5.888551,55.00,0.6,9.500000,10.600000,244.600000,...,176.0,32.213358,56044.598,0.0,134555.534,0.0,46408.696597,0.000000,137466.685807,0.000000


In [63]:
y_train = train_data[labels]
y_test = test_data[labels]
y_test.head()

,logFCS,rCSI,HDDS
10,2.833213,0.00,3.00
11,3.198673,0.00,2.00
12,2.995732,8.75,2.25
13,3.178054,0.00,2.00
15,3.433987,0.00,3.00


In [79]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [80]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
x_train = poly.fit_transform(x_train)
x_test =  poly.fit_transform(x_test)



In [81]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train[labels[2]])

# Compute model predictions for test data
pred = model.predict(x_test)

actual = y_test[labels[2]]
r2_linear = stats.pearsonr(actual, pred)[0] ** 2
r2_linear

0.0009861335487284884

In [66]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True)

parameters = {'max_depth':np.arange( 1,8, 1 ).tolist(), 'min_samples_leaf':np.arange( 1, 6, 1 ).tolist()}
clf = GridSearchCV(rfc, parameters,cv=6, n_jobs= 4, iid = True,  refit= True,pre_dispatch= '2*n_jobs')
clf.fit(x_train, y_train[labels[0]])

# Fit estimator and display score

# Regress on test data
pred = clf.predict(x_test)
actual = y_test[labels[2]]
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Administrator\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Administrator\Anacon

In [67]:
from sklearn.ensemble import ExtraTreesRegressor

# Create Regressor with default properties
extratree = ExtraTreesRegressor(random_state=0)

# Fit estimator and display score
extratree = extratree.fit(x_train, y_train[labels[2]])
# Regress on test data
pred = extratree.predict(x_test)

actual = y_test[labels[2]]
r2_et = stats.pearsonr(actual, pred)[0] ** 2
r2_et




0.0013496236653385944

In [14]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(warm_start=False,random_state =0, validation_fraction=0.1,learning_rate='adaptive')
# Fit estimator and display score
mlp = mlp.fit(x_train, y_train[labels[2]])
pred = mlp.predict(x_test)

actual = y_test[labels[2]]
r2_mlp= stats.pearsonr(actual, pred)[0] ** 2
r2_mlp

0.169946703220582

In [68]:
from sklearn.svm import SVR

# Create Regressor with default properties
svr = SVR(kernel='linear',degree =4)

# Fit estimator and display score
svr = svr.fit(x_train, y_train[labels[2]])
pred = svr.predict(x_test)

actual = y_test[labels[2]]
r2_svr= stats.pearsonr(actual, pred)[0] ** 2
r2_svr


0.03253303610022786

In [16]:
from sklearn.gaussian_process import  GaussianProcessRegressor

# Import our kernels
from sklearn.gaussian_process.kernels import Matern, WhiteKernel



# Define custom kernel (Matern + noise)
krnl = 2. + Matern(length_scale=1, nu=1.8) + WhiteKernel(noise_level=10)

# Create Regressor with specified properties
gpr = GaussianProcessRegressor(kernel=krnl, random_state=0)

# Fit estimator and display score
gpr= gpr.fit(x_train, y_train[labels[2]])
pred = gpr.predict(x_test)
actual = y_test[labels[2]]
r2_gpr= stats.pearsonr(actual, pred)[0] ** 2
r2_gpr



0.69498912222257903

In [82]:

from sklearn.linear_model import Ridge

# Define different alpha values for different fits
alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge = Ridge(alpha=20,fit_intercept=False)

ridge.fit(x_train, y_train[labels[2]])
pred = ridge.predict(x_test)

actual = y_test[labels[2]]
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

0.0010890826720968408

In [83]:
from sklearn.linear_model import BayesianRidge
bridge = BayesianRidge(compute_score=True)
bridge.fit(x_train, y_train[labels[2]])
pred = bridge.predict(x_test)

actual = y_test[labels[2]]
r2_bridge= stats.pearsonr(actual, pred)[0] ** 2
r2_bridge

4.8161044881285065e-05

In [88]:
from sklearn.linear_model import Lasso
alpha = [1E-4, 1E-2, 0.1, 1.0]

ls = Lasso(alpha=alpha[3], fit_intercept=True)


ls = ls.fit(x_train, y_train[labels[2]])
pred = ls.predict(x_test)

actual = y_test[labels[2]]
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

0.08401047638045903

In [86]:
from sklearn.linear_model import ElasticNet
alpha = [1E-4, 1E-2, 0.1, 1.0]

en = ElasticNet(alpha=alpha[2], l1_ratio=0.8)


en.fit(x_train, y_train[labels[2]])
pred = en.predict(x_test)

actual = y_test[labels[2]]
r2_en= stats.pearsonr(actual, pred)[0] ** 2
r2_en


0.036772545065601896

In [73]:
from sklearn.ensemble import GradientBoostingRegressor

# Create Regressor with default properties
gbr = GradientBoostingRegressor(random_state=0)

gbr.fit(x_train, y_train[labels[2]])
pred = gbr.predict(x_test)

actual = y_test[labels[2]]
r2_gbr= stats.pearsonr(actual, pred)[0] ** 2
r2_gbr


0.001861164562984472